# 1 - Music source separation

**Based on the Hybrid Demucs Colab tutorial**

This supports the Demucs source separation model (https://github.com/facebookresearch/demucs/)

## Setup

In [ ]:
%%capture
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [ ]:
from google.colab import drive
from google.colab import files
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Source separation

Define global settings

In [ ]:
model = "htdemucs"
extensions = ["mp3"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/content/drive/MyDrive/Máster/DLASP/Final/songs'
out_path = '/content/drive/MyDrive/Máster/DLASP/Final/songs_separated'

# Creates folder if exist
Path(out_path).mkdir(exist_ok=True)

In [ ]:
#@title Auxiliar functions
def find_files(in_path):
  out = []
  yet = list(Path(out_path).glob("htdemucs/*"))
  yet = [folder.name for folder in yet]
  for file in Path(in_path).iterdir():
    if file.suffix.lower().lstrip(".") in extensions:
      filename = '.'.join(file.name.split('.')[:-1])
      if filename not in yet:
        out.append(file)
  return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


### Perform separation

In [ ]:
separate()

Going to separate the files:
/content/drive/MyDrive/Máster/DLASP/Final/songs/33_poesia_de_guerra.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/34_walkman.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/35_el_regreso.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/36_desafio.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/37_rapvida_2002.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/38_el_club_de_los_olvidados.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/39_rapvida_2003.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/40_chico_problematico.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/41_rap_vida_2001.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/42_calles.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/43_gracias.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/44_el_sexto_mandamiento.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/45_ser_o_no_ser.mp3
/content/drive/MyDrive/Máster/DLASP/Final/songs/46_en_el_punto_de

Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:02<00:00, 35.6MB/s]

Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/drive/MyDrive/Máster/DLASP/Final/songs_separated/htdemucs
Separating track /content/drive/MyDrive/Máster/DLASP/Final/songs/33_poesia_de_guerra.mp3



100%|██████████████████████████████████████████████| 298.34999999999997/298.34999999999997 [00:20<00:00, 14.48seconds/s]

Separating track /content/drive/MyDrive/Máster/DLASP/Final/songs/34_walkman.mp3


...


100%|████████████████████████████████████████████████████████████████████████| 362.7/362.7 [00:17<00:00, 20.20seconds/s]

Separating track /content/drive/MyDrive/Máster/DLASP/Final/songs/116_grande.mp3



100%|██████████████████████████████████████████████████████████████████████| 239.85/239.85 [00:11<00:00, 20.10seconds/s]


Separating track /content/drive/MyDrive/Máster/DLASP/Final/songs/117_juega.mp3


100%|██████████████████████████████████████████████| 269.09999999999997/269.09999999999997 [00:13<00:00, 20.53seconds/s]


## Convert to wav

In [ ]:
%%capture

!pip install pydub

In [ ]:
from pydub import AudioSegment

In [ ]:
drive.mount('/content/drive')

songs_path = Path("drive/MyDrive/Máster/DLASP/Final/songs_separated/htdemucs")

In [ ]:
for folder in (pbar := tqdm(list(songs_path.iterdir()), leave=False)):
  pbar.set_description(folder.name)
  # convert mp3 to wav                                                 
  sound = AudioSegment.from_mp3(f'{folder}/vocals.mp3')
  sound.export(f'{folder}/vocals.wav', format="wav")